# Environment Configuration

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime
from itertools import product

from google.colab import auth
from google.cloud import bigquery
from gspread_dataframe import set_with_dataframe

import gspread
from oauth2client.client import GoogleCredentials
from google.auth import default
creds, _ = default()

import pandas_gbq
from tqdm import tqdm
import time
from datetime import datetime
from shapely.geometry import Point, Polygon
import time
import random
import json

auth.authenticate_user()
print('Authenticated')

# Project Configuration
project_id = 'foodpanda-tw-bigquery'
client = bigquery.Client(project=project_id)

In [ ]:
start_datetime = datetime.now()
print(f"Script started at: {start_datetime.strftime('%Y-%m-%d %H:%M:%S')}")
start_time = time.time()

# Mapbox API Test Area



In [ ]:
import requests
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import shape, Point

# insert api key
API_KEY = 'token_name'

# target_point
coords = [121.5654, 25.0330]  # Taipei 101

# add point(longitude, latitude), drive time and api key
url = f"https://api.mapbox.com/isochrone/v1/mapbox/cycling/{coords[0]},{coords[1]}?contours_minutes=15&access_token={API_KEY}"

# send requests
response = requests.get(url)
data = response.json()

# print response
print(data)

# Upload point_df

In [ ]:
# Points Dataframe

point_df = client.query(f'''

SELECT DISTINCT
  longitude,
  latitude,
FROM zone_locations
WHERE lg_city_name IN ("Taitung")

''').to_dataframe()

point_df

# Main Script

In [ ]:
import requests
import geopandas as gpd
from shapely.geometry import LineString, Polygon
import pandas as pd

# Add Mapbox API key
API_KEY = 'token_name'

# Drive time range
minutes_range = range(26, 46)

# Initialize final results list
results = []

for idx, row in point_df.iterrows():
    coords = [row['longitude'], row['latitude']]

    for mins in minutes_range:
        url = f"https://api.mapbox.com/isochrone/v1/mapbox/cycling/{coords[0]},{coords[1]}?contours_minutes={mins}&access_token={API_KEY}"
        response = requests.get(url)
        data = response.json()

        # Print status
        # print(f"Fetching data for {mins} minutes for point {coords}...")

        # Check if the response contains 'features'
        if 'features' in data and len(data['features']) > 0:
            geometry = data['features'][0]['geometry']
            if geometry['type'] == 'LineString':  # Change to 'Polygon'
                coordinates = geometry['coordinates']  # Access outer ring of the Polygon
                line = LineString(coordinates)
                polygon = Polygon(coordinates).buffer(0.001)

                # Append the valid polygon data to results
                results.append({
                    'point_latitude': coords[1],
                    'point_longitude': coords[0],
                    'mins': mins,
                    'polygon': polygon
                })
            else:
                print(f"Expected a Polygon geometry type, but got {geometry['type']}.")
        else:
            print("No 'features' key in the response or invalid response format.")

# Convert results to GeoDataFrame
df = pd.DataFrame(results)

print("The data has been successfully saved to the DataFrame.")


In [ ]:
df['polygon'] = df['polygon'].astype(str)
df

In [ ]:
df.dtypes

# Upload dataframe to GCP

In [ ]:
!pip install pandas_gbq==0.20.0

In [ ]:
import pandas as pd
from datetime import datetime
import json
import google.oauth2.credentials
import pandas_gbq

In [ ]:
with open('/content/drive/MyDrive/user_account.json') as f:
    credential = json.load(f)

credentials = google.oauth2.credentials.Credentials(
    credential['token'],
    refresh_token=credential['refresh_token'],
    token_uri=credential['token_uri'],
    client_id=credential['client_id'],
    client_secret=credential['client_secret']
)

In [ ]:
pandas_gbq.context.credentials = credentials
pandas_gbq.context.project = 'database_name'

project_id = "database_name"
table_id = 'table_name'

df.to_gbq(
    destination_table=table_id,
    project_id=project_id,
    credentials=credentials,
    if_exists='append'
)

print(f"Data has been uploaded to BigQuery table '{table_id}'.")

In [ ]:
end_time = time.time()
execution_time_seconds = end_time - start_time
execution_time_minutes = execution_time_seconds / 60
print(f"Script completed in {execution_time_minutes:.2f} minutes.")